In [1]:
import mysql.connector
import matplotlib.pyplot as plt
import seaborn as sns
import csv
import os
import pandas as pd

%matplotlib inline



In [2]:
db_connection = mysql.connector.connect(
    host="localhost",
    user= "root",
    passwd = "123456484848t", #change password
    auth_plugin = 'mysql_native_password'
)
print(db_connection)



In [3]:
db_cursor = db_connection.cursor(buffered=True)
db_cursor.execute("SET GLOBAL innodb_lock_wait_timeout = 120")

In [4]:
db_cursor.execute("CREATE DATABASE IF NOT EXISTS fitness_tracker_db")

In [5]:
db_cursor.execute("USE fitness_tracker_db")

In [6]:
def populate_table(db_connection, db_cursor, insert_query, file_path, batch_size=100):
    import csv
    with open(file_path, mode='r') as csv_data:
        reader = csv.reader(csv_data, delimiter=',')
        csv_data_list = list(reader)
        for i, row in enumerate(csv_data_list[1:], start=1):  # Skip the header
            row = tuple(map(lambda x: None if x == "" else x, row))  # Convert empty strings to None
            db_cursor.execute(insert_query, row)  # Execute the insert query
            if i % batch_size == 0:  # Commit every `batch_size` rows
                db_connection.commit()
        db_connection.commit()  # Final commit after all rows are processed

In [7]:
db_cursor.execute("""CREATE TABLE IF NOT EXISTS User (
        id CHAR(8),
        email VARCHAR(320) NOT NULL,
        password VARCHAR(255) NOT NULL,
        fname VARCHAR(30) NOT NULL,
        lname VARCHAR(30) NOT NULL,
        age TINYINT UNSIGNED CHECK (age > 0 AND age <= 120),
        gender CHAR(1) CHECK (gender IN ('M', 'F', 'O')),
        weight DECIMAL(5,2) CHECK (weight > 0),
        height DECIMAL(4,1) CHECK (height > 0),
        PRIMARY KEY (id))""")

insert_member= (
    "INSERT INTO User(id, email, password, fname, lname, age, gender, weight, height)"
    "VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)"
)


file_path = os.path.expanduser("~/Desktop/csv_files/users.csv")  # Path to the CSV file
users_df = pd.read_csv(file_path)  # Read CSV into a DataFrame (optional for analysis, not used here)

# Call the populate_table function to insert data
populate_table(db_connection, db_cursor, insert_member, file_path, batch_size=50)


print("Completed")

Completed


In [8]:
db_cursor.execute("""
CREATE TABLE IF NOT EXISTS Gym (
    id CHAR(8),
    name VARCHAR(100),
    location VARCHAR(255),
    price DECIMAL(8,2) CHECK (price > 0),
    PRIMARY KEY (id)
)""")


insert_gym = (
    "INSERT INTO Gym (id, name, location, price)"
    "VALUES (%s, %s, %s, %s)"
)


file_path = os.path.expanduser("~/Desktop/csv_files/gyms.csv")
gyms_df = pd.read_csv(file_path, encoding='utf-8-sig')


populate_table(db_connection, db_cursor, insert_gym, file_path, batch_size=50)

print("Completed")
    

Completed


In [9]:
db_cursor.execute("""
CREATE TABLE IF NOT EXISTS RegularUser (
    id CHAR(8),
    PRIMARY KEY(id),
    FOREIGN KEY (id) REFERENCES User(id) ON DELETE CASCADE
)""")


insert_regular_user = (
    "INSERT INTO RegularUser (id)"
    "VALUES (%s)"
)

# File path for RegularUser CSV
file_path = os.path.expanduser("~/Desktop/csv_files/regular_users.csv")


populate_table(db_connection, db_cursor, insert_regular_user, file_path, batch_size=50)

print("Completed")



Completed


In [10]:
db_cursor.execute("""
CREATE TABLE IF NOT EXISTS PersonalTrainer (
    id CHAR(8) PRIMARY KEY,
    experience INT NOT NULL,
    specialization VARCHAR(255) NOT NULL,
    FOREIGN KEY (id) REFERENCES User(id) ON DELETE CASCADE
)""")

insert_personal_trainer = (
    "INSERT INTO PersonalTrainer (id, experience, specialization)"
    "VALUES (%s, %s, %s)"
)

# File path for PersonalTrainer CSV
file_path = os.path.expanduser("~/Desktop/csv_files/trainers.csv")

# Read the CSV file
personal_trainers_df = pd.read_csv(file_path)


# Populate the PersonalTrainer table
populate_table(db_connection, db_cursor, insert_personal_trainer, file_path, batch_size=50)


print("Completed")

Completed


In [12]:
db_cursor.execute("""
CREATE TABLE IF NOT EXISTS Have_Membership_AT (
    user_id CHAR(8),
    gym_id CHAR(8),
    start_date DATE NOT NULL,
    due_date DATE NOT NULL,
    PRIMARY KEY (user_id, gym_id),
    FOREIGN KEY (user_id) REFERENCES User(id) ON DELETE CASCADE,
    FOREIGN KEY (gym_id) REFERENCES Gym(id) ON DELETE CASCADE
)""")



insert_membership = (
    "INSERT INTO Have_Membership_AT (user_id, gym_id, start_date, due_date)"
    "VALUES (%s, %s, %s, %s)"
)



# File path for Have_Membership_AT CSV
file_path = os.path.expanduser("~/Desktop/csv_files/memberships.csv")


# Populate the Have_Membership_AT table
populate_table(db_connection, db_cursor, insert_membership, file_path, batch_size=50)

print("Completed")

Completed


In [13]:
db_cursor.execute("SET FOREIGN_KEY_CHECKS = 0");
db_cursor.execute("""
CREATE TABLE IF NOT EXISTS Workout (
    id CHAR(8),
    gym_id CHAR(8) NOT NULL,
    date DATE NOT NULL,
    start_time TIME NOT NULL,
    end_time TIME NOT NULL,
    trainer_id CHAR(8),
    reg_user_id CHAR(8),
    PRIMARY KEY(id),
    FOREIGN KEY (gym_id) REFERENCES Gym(id) ON DELETE CASCADE,
    FOREIGN KEY (trainer_id) REFERENCES PersonalTrainer(id) ON DELETE CASCADE,
    FOREIGN KEY (reg_user_id) REFERENCES RegularUser(id) ON DELETE CASCADE
)
""")


insert_workout = (
    "INSERT INTO Workout (id, gym_id, date, start_time, end_time, trainer_id, reg_user_id)"
    "VALUES (%s, %s, %s, %s, %s, %s, %s)"
)


# File path for Workouts CSV
file_path = os.path.expanduser("~/Desktop/csv_files/workouts.csv")

populate_table(db_connection, db_cursor, insert_workout, file_path, batch_size=50)

print("Completed")


Completed


In [14]:
db_cursor.execute("""
CREATE TABLE IF NOT EXISTS Exercise (
    name VARCHAR(100) PRIMARY KEY,
    required_equipment VARCHAR(100),
    level ENUM('Beginner', 'Intermediate', 'Advanced') NOT NULL
)""")


insert_exercise = (
    "INSERT INTO Exercise (name, required_equipment, level)"
    "VALUES (%s, %s, %s)"
)






# File path for Exercise CSV
file_path = os.path.expanduser("~/Desktop/csv_files/exercises.csv")



# Populate the Exercise table
populate_table(db_connection, db_cursor, insert_exercise, file_path, batch_size=50)


print("Completed")


Completed


In [15]:
db_cursor.execute("SET FOREIGN_KEY_CHECKS = 0");
db_cursor.execute("""
CREATE TABLE IF NOT EXISTS Performance (
    id CHAR(8),
    weight DECIMAL(5,2) CHECK (weight >= 0),
    sets TINYINT UNSIGNED CHECK (sets > 0),
    reps TINYINT UNSIGNED CHECK (reps > 0),
    workout_id CHAR(8),
    exercise_name VARCHAR(100) NOT NULL,
    PRIMARY KEY(id),
    FOREIGN KEY (workout_id) REFERENCES Workout(id) ON DELETE CASCADE,
    FOREIGN KEY (exercise_name) REFERENCES Exercise(name) ON DELETE CASCADE
)""")


insert_performance = (
    "INSERT INTO Performance (id, weight, sets, reps, workout_id, exercise_name)"
    "VALUES (%s, %s, %s, %s, %s, %s)"
)



# Save valid data to CSV
file_path = os.path.expanduser("~/Desktop/csv_files/performances.csv")



populate_table(db_connection, db_cursor, insert_performance, file_path, batch_size=50)

print("Completed")

Completed


In [16]:
db_cursor.execute("""
CREATE TABLE IF NOT EXISTS Target_Muscle (
    exercise_name VARCHAR(100),
    muscle VARCHAR(50),
    PRIMARY KEY (exercise_name, muscle),
    FOREIGN KEY (exercise_name) REFERENCES Exercise(name)
)""")

insert_target_muscle = (
    "INSERT INTO Target_Muscle (exercise_name, muscle)"
    "VALUES (%s, %s)"
)


# File path for Target_Muscle CSV
file_path = os.path.expanduser("~/Desktop/csv_files/target_muscles.csv")


# Populate the Target_Muscle table
populate_table(db_connection, db_cursor, insert_target_muscle, file_path, batch_size=50)

print("Completed")




Completed
